In [1]:
# Importing the libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
tf.__version__

'2.17.0'

In [3]:
#preprocessing the training set
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,
                                   zoom_range = 0.2,horizontal_flip = True)
training_set = train_datagen.flow_from_directory('datasets/training_set',target_size = (64, 64),
                                                 batch_size = 32,class_mode = 'binary')

Found 8000 images belonging to 2 classes.


In [4]:
#preprocessing test dataset
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('datasets/test_set',target_size = (64, 64),
                                            batch_size = 32,class_mode = 'binary')

Found 2000 images belonging to 2 classes.


In [5]:
cnn = tf.keras.models.Sequential()

# Convolution Layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

#Pooling Layer
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

#Flatenning
cnn.add(tf.keras.layers.Flatten())

cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

#Output Layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

d:\ML_ENV\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

import datetime
early_stopping = EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

cnn.fit(x = training_set, validation_data = test_set, epochs = 25, callbacks=[early_stopping, tensorboard_callback])

Epoch 1/75


d:\ML_ENV\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 351s 1s/step - accuracy: 0.5227 - loss: 0.6897 - val_accuracy: 0.6135 - val_loss: 0.7178
Epoch 2/75
250/250 ━━━━━━━━━━━━━━━━━━━━ 34s 134ms/step - accuracy: 0.6732 - loss: 0.6055 - val_accuracy: 0.7300 - val_loss: 0.5448
Epoch 3/75
250/250 ━━━━━━━━━━━━━━━━━━━━ 34s 134ms/step - accuracy: 0.7178 - loss: 0.5567 - val_accuracy: 0.7330 - val_loss: 0.5347
Epoch 4/75
250/250 ━━━━━━━━━━━━━━━━━━━━ 35s 135ms/step - accuracy: 0.7245 - loss: 0.5425 - val_accuracy: 0.7590 - val_loss: 0.5010
Epoch 5/75
250/250 ━━━━━━━━━━━━━━━━━━━━ 35s 136ms/step - accuracy: 0.7539 - loss: 0.5014 - val_accuracy: 0.7480 - val_loss: 0.5113
Epoch 6/75
250/250 ━━━━━━━━━━━━━━━━━━━━ 31s 121ms/step - accuracy: 0.7585 - loss: 0.4897 - val_accuracy: 0.7490 - val_loss: 0.5138
Epoch 7/75
250/250 ━━━━━━━━━━━━━━━━━━━━ 29s 114ms/step - accuracy: 0.7702 - loss: 0.4736 - val_accuracy: 0.7720 - val_loss: 0.4933
Epoch 8/75
250/250 ━━━━━━━━━━━━━━━━━━━━ 30s 116ms/step - accuracy: 0.7832 - loss: 0.4589 - val_a

In [7]:
'''
import numpy as np
from tensorflow.keras.preprocessing import image


test_image = image.load_img('PATH_TO_FILE', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

'''


"\nimport numpy as np\nfrom tensorflow.keras.preprocessing import image\n\n\ntest_image = image.load_img('PATH_TO_FILE', target_size = (64, 64))\ntest_image = image.img_to_array(test_image)\ntest_image = np.expand_dims(test_image, axis = 0)\nresult = cnn.predict(test_image)\ntraining_set.class_indices\nif result[0][0] == 1:\n  prediction = 'dog'\nelse:\n  prediction = 'cat'\n\n"

In [8]:
cnn.save("models/cnn.keras")

In [9]:
%load_ext tensorboard
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6008 (pid 8936), started 3 days, 16:02:16 ago. (Use '!kill 8936' to kill it.)